In [ ]:
import os
os.chdir("..")

In [ ]:
import numpy as np
import spreg
import libpysal
import bambi as bmb
import arviz as az
import pandas as pd
import geopandas as gpd
import numpy
from pysal.lib import weights
import polars as pl
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from pysal.lib import cg as geometry

from spreg import dgp_lag

from src.data.data_reg import SpatialReg

sr = SpatialReg(n=10)


In [ ]:
gdf = sr.spatial_panel(n=10,time=4,rho=.8)
gdf

In [ ]:
gdf.plot("X1")

In [ ]:
gdf.plot("y_d")

In [ ]:
wr = weights.contiguity.Rook.from_dataframe(gdf[gdf["time"] == 0])
wr.transform = "r"
y_d = gdf["y_d"].values.reshape(-1,1)
xb = gdf["X1"].values.reshape(-1,1)
fe_lag = spreg.Panel_FE_Lag(y=y_d, x=xb, w=wr)
print(fe_lag.summary)

In [ ]:
df = gdf.drop("geometry", axis=1)
model = bmb.Model(
    "y_d ~ 1 + X1 + w_d",
    df, dropna=True
)
results = model.fit()

In [ ]:
az.plot_trace(
    results,
    compact=True,
)

In [ ]:
az.summary(results)

In [ ]:
pl.read_csv("sim_results.csv")